In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/med-vit/MedViT.py
/kaggle/input/med-vit/utils.py
/kaggle/input/model-checkpoints/MedViT_small_im1k.pth


In [2]:
!pip install medmnist timm einops


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=c2c85a4cbd81e7e33cd0006a0da8ac8703706d9bc54bd94bf012c625c30f9748
  Stored in directory: /root/.cache/pip/wheels/19/39/2f/2d3cadc408a8804103f1c34ddd4b9f6a93497b11fa96fe738e
Successfully built fire


In [3]:
import sys
sys.path.append('/kaggle/input/med-vit')

from MedViT import MedViT_small  # or your class/function name


/usr/local/lib/python3.10/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/usr/local/lib/python3.10/dist-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)


In [4]:
model = MedViT_small(num_classes=13)
print(model)


initialize_weights...
MedViT(
  (stem): Sequential(
    (0): ConvBNReLU(
      (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): ReLU(inplace=True)
    )
    (1): ConvBNReLU(
      (conv): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): ReLU(inplace=True)
    )
    (2): ConvBNReLU(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): ReLU(inplace=True)
    )
    (3): ConvBNReLU(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (

In [5]:
from medmnist import BreastMNIST, RetinaMNIST, OCTMNIST, PneumoniaMNIST
from torchvision import transforms
from torch.utils.data import ConcatDataset

transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to match model input
    transforms.Grayscale(num_output_channels=3),  # Convert to 3-channel
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5], std=[.5])
])

# Load datasets
datasets = []
label_offset = 0
class_counts = []

for dataset_class, num_classes in [(BreastMNIST, 2), (RetinaMNIST, 5), (OCTMNIST, 4), (PneumoniaMNIST, 2)]:
    ds = dataset_class(split='train', transform=transform, download=True)
    
    # Adjust labels to avoid collisions
    ds.targets = ds.labels + label_offset
    datasets.append(ds)
    
    label_offset += num_classes
    class_counts.append(num_classes)

# Merge into one dataset
combined_dataset = ConcatDataset(datasets)


100%|██████████| 560k/560k [00:00<00:00, 747kB/s] 


100%|██████████| 3.29M/3.29M [00:01<00:00, 3.08MB/s]


100%|██████████| 54.9M/54.9M [00:05<00:00, 10.9MB/s]


100%|██████████| 4.17M/4.17M [00:01<00:00, 3.71MB/s]


In [6]:
from torch.utils.data import DataLoader

train_loader = DataLoader(combined_dataset, batch_size=64, shuffle=True)


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.amp import autocast, GradScaler


model = MedViT_small(num_classes=13)  # <- important
state_dict = torch.load('/kaggle/input/model-checkpoints/MedViT_small_im1k.pth', map_location='cpu')['model']
state_dict = {k: v for k, v in state_dict.items() if not k.startswith('proj_head.0')}
model.load_state_dict(state_dict, strict=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
num_epochs = 10

scaler = GradScaler()

model.train()
for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in train_loader:
        images = images.to(device).float()
        labels = labels.to(device).squeeze().long()  # for CrossEntropyLoss

        optimizer.zero_grad()

        with autocast(device_type='cuda'):  # enables mixed precision
            outputs = model(images)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}] Loss: {running_loss:.4f}")



initialize_weights...


<ipython-input-7-f91c1689f3a2>:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load('/kaggle/input/model-checkpoints/MedViT_small_im1k.pth', map_location

Epoch [1/10] Loss: 594.7213
Epoch [2/10] Loss: 415.9829
Epoch [3/10] Loss: 359.6416
Epoch [4/10] Loss: 312.0146
Epoch [5/10] Loss: 266.6064
Epoch [6/10] Loss: 229.1824
Epoch [7/10] Loss: 187.1830
Epoch [8/10] Loss: 158.0093
Epoch [9/10] Loss: 138.0147
Epoch [10/10] Loss: 110.7684


In [8]:
torch.save({
    'epoch': num_epochs,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict()
}, '/kaggle/working/medvit_checkpoint.pth')


In [9]:
# Re-initialize model and optimizer
model = MedViT_small(num_classes=13)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
model.to(device)

# Load checkpoint
checkpoint = torch.load('/kaggle/working/medvit_checkpoint.pth')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

start_epoch = checkpoint['epoch'] + 1  # Resume from next epoch
new_epochs = 10

scaler = GradScaler()

model.train()
for epoch in range(start_epoch, start_epoch + new_epochs):
    running_loss = 0.0
    for images, labels in train_loader:
        images = images.to(device).float()
        labels = labels.to(device).squeeze().long()

        optimizer.zero_grad()
        with autocast(device_type='cuda'):
            outputs = model(images)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{start_epoch + new_epochs - 1}] Loss: {running_loss:.4f}")


initialize_weights...


<ipython-input-9-f959a029bd61>:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('/kaggle/working/medvit_checkpoint.pth')


Epoch [12/20] Loss: 104.8400
Epoch [13/20] Loss: 95.5870
Epoch [14/20] Loss: 83.1757
Epoch [15/20] Loss: 80.2658
Epoch [16/20] Loss: 69.1438
Epoch [17/20] Loss: 67.7199
Epoch [18/20] Loss: 64.8027
Epoch [19/20] Loss: 59.3029
Epoch [20/20] Loss: 53.1500
Epoch [21/20] Loss: 57.7799


In [10]:
torch.save({
    'epoch': num_epochs,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict()
}, '/kaggle/working/medvit_checkpoint_two.pth')
